# Hyperparamter tuning

## Objetivos
*  
*

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import regularizers

import keras_tuner as kt

import numpy as np
import pandas as pd
from IPython import display
from matplotlib import pyplot as plt

import os

print(tf.__version__)

2.18.0


## Importación y pretratamiento de datos
En esta demostración se va a utilizar de nuevo la base de datos *Wine Quality*, disponible en [UC Irvine Machine Learning Repository](https://archive.ics.uci.edu/dataset/186/wine+quality).

In [ ]:
if 'google.colab' in str(get_ipython()):
  data_path = 'https://raw.githubusercontent.com/cursos-COnCEPT/curso-tensorflow/refs/heads/main/winequality-red.csv'
else:
  data_path = os.getcwd() + '\\winequality-red.csv'

red_wine = pd.read_csv(data_path, sep=';')
red_wine.tail()

# Split data
train_ratio = .7
val_ratio = .15
df_train = red_wine.sample(frac=train_ratio+val_ratio, random_state=0)
df_test = red_wine.drop(df_train.index)
df_valid = red_wine.sample(frac=val_ratio/(val_ratio+train_ratio), random_state=0)

# Split features and target
X_train = df_train.drop('quality', axis=1)
X_valid = df_valid.drop('quality', axis=1)
X_test = df_test.drop('quality', axis=1)
y_train = df_train['quality']
y_valid = df_valid['quality']
y_test = df_test['quality']

# Scale to [0, 1]
max_ = X_train.max(axis=0)
min_ = X_train.min(axis=0)
X_train = (X_train - min_) / (max_ - min_)
X_valid = (X_valid - min_) / (max_ - min_)
X_test = (X_test - min_) / (max_ - min_)



## Definición del modelo

Cuando se construye un modelo para *hypertuning*, se definen tanto el espacio de búsqueda de hiperparámetros como la arquitectura del modelo. El modelo que configure para el hipertuning se denomina *hypermodel*.

Aunque puede definirse un *hypermodel*, aquí lo haremos utilizando una función que sirva para construir el modelo. Esta función devolverá un modelo ya compilado donde que utiliza hiperparámetros en lugar de valores fijos para aquellos elementos que queramos "tunear".

A continuación, ajusta el número de neuronas de la primera capa, que puede estar comprendido entre 16 y 128, variando en potencias de base 2. Además, escoge el valor óptimo para el learning rate entre las siguientes opciones [0.001, 0.005, 0.0001].

In [3]:
def model_builder(hp):
    hp_neurons = hp.Int('neurons', 
                        min_value=16, 
                        max_value=128, 
                        step=2, 
                        sampling='log')
    hp_learning_rate = hp.Choice('learning_rate', 
                                values=[1e-3, 5e-4, 1e-4])

    model = keras.Sequential([
        layers.Input(shape=[X_train.shape[1]]),
        layers.Dense(hp_neurons, activation='relu'),
        layers.Dense(16, activation='relu'),
        layers.Dense(1),
    ])

    optimizer = tf.keras.optimizers.Adam(hp_learning_rate)
    
    model.compile(optimizer=optimizer,loss='mae')

    return model

## Ajuste de hiperparámetros

En Keras hay cuatro métodos diferentes para realizar la búsqueda - `RandomSearch`, `Hyperband`, `BayesianOptimization`, y `Sklearn`. En este ejemplo se utiliza el método `Hyperband`. 

Este algoritmo es rápido y eficiente porque prueba varias combinaciones de opciones (como el número de capas o el *learning rate*) y asigna más recursos (tiempo de entrenamiento) a las opciones que están funcionando mejor, para así encontrar la mejor solución en menos tiempo.

Para instanciar el *tuner*, hay que especificar el hipermodelo, el objetivo a optimizar y el número máximo de épocas a entrenar. 

In [10]:
tuner = kt.Hyperband(model_builder,
                     objective='val_loss',
                     max_epochs=100,
                     overwrite=True)
                     
tuner.search_space_summary()

Search space summary
Default search space size: 2
neurons (Int)
{'default': None, 'conditions': [], 'min_value': 16, 'max_value': 128, 'step': 2, 'sampling': 'log'}
learning_rate (Choice)
{'default': 0.001, 'conditions': [], 'values': [0.001, 0.0005, 0.0001], 'ordered': True}


Crea un callback tipo `EarlyStopping` antes de efectuar la búsqueda.

In [11]:
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(
    monitor='val_loss', # what to track
    min_delta=0.001, # minimium amount of change to count as an improvement
    patience=20, # how many epochs to wait before stopping
    restore_best_weights=True,
)

Efectúa la búsqueda, conocida como *hyperparameter tuning* con el comando `tuner.search`. Esta función utiliza los mismos argumentos que el método `model.fit`.

In [12]:
tuner.search(X_train, y_train,
            epochs=100,
            validation_data=(X_valid, y_valid), 
            batch_size=256,
            callbacks=[early_stopping],
            verbose='auto')

# Extraer la mejor combinación de hiperparámetros
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
El número de neuronas óptimo en la primera capa es {best_hps.get('neurons')} \n y el valor óptimo para el learning rate es {best_hps.get('learning_rate')}.
""")

Trial 12 Complete [00h 00m 03s]
val_loss: 5.527632713317871

Best val_loss So Far: 4.859363079071045
Total elapsed time: 00h 00m 35s

El número de neuronas óptimo en la primera capa es 64 
 y el valor óptimo para el learning rate es 0.001.



## Entrenamiento y evaluación del modelo

Crea una nueva instancia del modelo utilizando la mejor configuración de los hiperparámetros y lleva a cabo el entrenamiento.

In [13]:
model = tuner.hypermodel.build(best_hps)
history = model.fit(X_train, y_train, 
                    epochs=200,
                    validation_data=(X_valid, y_valid), 
                    batch_size=256,
                    callbacks=[early_stopping],
                    verbose='auto')

Epoch 1/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 2s 90ms/step - loss: 5.6243 - val_loss: 5.4588
Epoch 2/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - loss: 5.4552 - val_loss: 5.3171
Epoch 3/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - loss: 5.3022 - val_loss: 5.1665
Epoch 4/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - loss: 5.1596 - val_loss: 4.9897
Epoch 5/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 4.9572 - val_loss: 4.7672
Epoch 6/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 4.7297 - val_loss: 4.4893
Epoch 7/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 4.4246 - val_loss: 4.1464
Epoch 8/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - loss: 4.0940 - val_loss: 3.7052
Epoch 9/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 3.6068 - val_loss: 3.1278
Epoch 10/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 2.9660 - val_loss: 2.4008
Epoch 11/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 2.2215 - val_loss: 1.5592
Epoch 12/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - loss: 1.3674 - val_lo

A continuación, evalúa la calidad del ajuste del modelo.

In [14]:
eval_result = model.evaluate(X_test, y_test)
print("MAE:", eval_result)

8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.4514 
MAE: 0.4729171693325043


## Conclusiones
Este ejercicio no ha sido más que una introducción muy breve al tuneado de hiperparámetros. Para aprender más sobre el Keras Tuner, échale un vistazo a estos recursos:

* [Keras Tuner en el blog de TensorFlow](https://blog.tensorflow.org/2020/01/hyperparameter-tuning-with-keras-tuner.html)
* [Sitio web del Sintonizador Keras](https://keras-team.github.io/keras-tuner/)